# Bicep Curl Counter

This project utilizes Mediapipe for real-time pose estimation, enabling the calculation of the angle between the left shoulder, elbow, and wrist to monitor the count of bicep curls. The system also provides visual feedback on the arm status, showing 'down' when relaxed and 'up' when curled.

### Install and import Dependencies

In [1]:
!pip install mediapipe opencv-python

In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose

### Calculate angle

In [3]:
def calculate_angle(a,b,c):
    a=np.array(a) #First
    b=np.array(b) #Mid
    c=np.array(c) #End
    
    radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)
    
    if angle>180.0:
        angle=360-angle
    
    return angle

### Curl Counter

In [4]:
cap=cv2.VideoCapture(0)

#Curl Counter variables
counter = 0
stage = None

with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame=cap.read()
        
        #Recolor image because when we get a feed using opencv its in BGR format but we want it in RGB format
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable=False
        
        #make detections
        results=pose.process(image)
        
        #Recolor back to BGR format
        image.flags.writeable=True
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        #Extract Landmarks
        try:
            landmarks=results.pose_landmarks.landmark
            
            #Get Coordinates
            shoulder=[landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow=[landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist=[landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            #Calculate angle
            angle=calculate_angle(shoulder,elbow,wrist)
            
            #Visualise
            cv2.putText(image,str(angle),
                        tuple(np.multiply(elbow,[1280,740]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),2,cv2.LINE_AA)
        
            #Curl Counter logic
            if angle>160:
                stage="down"
            if angle<30 and stage=='down':
                stage="up"
                counter+=1

        except:
            pass
        
        #Render Curl Counter
        #Setup status box
        cv2.rectangle(image,(0,0),(200,180),(224, 45, 194),-1)
        
        #Rep data
        cv2.putText(image,'REPS',(15,25),
                    cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,str(counter),(10,85),
                    cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(image,stage,(10,145),
                    cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2,cv2.LINE_AA)
        
        #Render Detections
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                                 )
         
        cv2.imshow('Mediapipe Feed',image)
    
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()


I0000 00:00:1703443707.799019       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
